# Utilities

In [ ]:
from __future__ import division
import tensorflow as tf
from tensorflow.contrib.layers import *
import numpy as np
import random
from tensorflow.python.training.adam import AdamOptimizer


def ndmatmul(A, B):
    get_shape = lambda T, i: T.get_shape()[i] if T.get_shape()[i].value != None else -1
    X = tf.reshape(A, shape=[-1, get_shape(A, -1)])
    return tf.reshape(tf.matmul(X, B), shape=[get_shape(A, 0), get_shape(A, 1), get_shape(B, 1)])


def partition(l, k):
    return [l[i:i + k] for i in range(0, len(l), k)]

# Computational Graph

In [ ]:
class RNN:
    def __init__(self):
        tf.set_random_seed(1)
        N = 10
        L = self.L = 5
        self.input = tf.placeholder(dtype=tf.float32, shape=[None, L, 1])
        self.desired = tf.placeholder(dtype=tf.float32, shape=[None, L, 1])
        self.prev_state = tf.placeholder(dtype=tf.float32, shape=[None, N])
        xv = xavier_initializer()
        ones = tf.initializers.ones()
        self.rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=N, activation=tf.nn.leaky_relu, name="" + str(
            random.random()))  # ,kernel_initializer=xv,bias_initializer=ones)
        self.wy = tf.Variable(xv(shape=[N, 1]), dtype=tf.float32)
        self.by = tf.Variable(ones(shape=[1, 1]), dtype=tf.float32)
        self.hidden_acts, self.states = tf.nn.dynamic_rnn(self.rnn_cell, inputs=self.input,
                                                          initial_state=self.prev_state)  # None x L x N
        self.out = tf.nn.leaky_relu(ndmatmul(self.hidden_acts, self.wy) + self.by)
        self.loss = tf.reduce_sum((self.out - self.desired) ** 2)
        opt = tf.train.AdamOptimizer()
        grads = opt.compute_gradients(self.loss)
        clipped = zip(tf.clip_by_global_norm([grad for grad, var in grads if grad != None], 10)[0],
                      [var for grad, var in grads if grad != None])
        self.train_ = opt.apply_gradients(clipped)
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

# Train the Model

In [ ]:
def train(self):
    L = self.L
    sess = self.sess
    X = []
    Y = []
    for i in range(2 ** L):
        x = bin(i)[2:]
        x = "0" * (L - len(x)) + x
        x = list(map(int, x))
        y = [x[0]]
        for i in x[1:]:
            y.append(y[-1] * 2 + i)
        X.append(x)
        Y.append(y)
    indices = list(range(1, 2 ** L))
    random.shuffle(indices)
    X = [X[i] for i in indices]
    Y = [Y[i] for i in indices]
    Xs = partition(X, 50)
    Ys = partition(Y, 50)

    zero_state_0 = self.rnn_cell.zero_state(batch_size=len(Xs[0]), dtype=tf.float32).eval(session=sess)
    zero_state_1 = self.rnn_cell.zero_state(batch_size=len(Xs[-1]), dtype=tf.float32).eval(session=sess)
    for epoch in range(1000):
        total_loss = 0
        for X, Y in zip(Xs, Ys):
            zero_state = zero_state_0 if len(X) == len(Xs[0]) else zero_state_1
            _, loss, curr_states = sess.run([self.train_, self.loss, self.states],
                                            feed_dict={self.input: np.array(X).reshape([-1, L, 1]),
                                                       self.desired: np.array(Y).reshape([-1, L, 1]),
                                                       self.prev_state: zero_state})
            total_loss += loss
        if epoch % 100 == 0:
            mean_loss = total_loss / 2 ** L
            print mean_loss

# Run the Computational Graph

In [ ]:
def run(self, seq):
    sess = self.sess
    L = self.L
    outputs = []
    prev_state = self.rnn_cell.zero_state(batch_size=1, dtype=tf.float32).eval(session=sess)
    while len(seq) >= L:
        s = seq[:L]
        inp = list(map(int, s))
        out, prev_state = self.sess.run([self.out, self.states],
                                        feed_dict={self.input: np.array(inp).reshape([1, -1, 1]),
                                                   self.prev_state: prev_state})
        outputs.extend(out.tolist()[0][:L])
        seq = seq[L:]
    if len(seq) > 0:
        s = seq + "0" * (L - len(seq))
        inp = list(map(int, s))
        out = self.sess.run([self.out],
                            feed_dict={self.input: np.array(inp).reshape([1, -1, 1]), self.prev_state: prev_state})[
            0]
        outputs.extend(out.tolist()[0][:len(seq)])
    return outputs


rnn = RNN()
rnn.train()
pass

In [2]:
from __future__ import division
import tensorflow as tf
from tensorflow.contrib.layers import *
import numpy as np
import random
from tensorflow.python.training.adam import AdamOptimizer


def ndmatmul(A, B):
    get_shape = lambda T, i: T.get_shape()[i] if T.get_shape()[i].value != None else -1
    X = tf.reshape(A, shape=[-1, get_shape(A, -1)])
    return tf.reshape(tf.matmul(X, B), shape=[get_shape(A, 0), get_shape(A, 1), get_shape(B, 1)])


def partition(l, k):
    return [l[i:i + k] for i in range(0, len(l), k)]


class RNN:
    def __init__(self):
        tf.set_random_seed(1)
        N = 10
        L = self.L = 5
        self.input = tf.placeholder(dtype=tf.float32, shape=[None, L, 1])
        self.desired = tf.placeholder(dtype=tf.float32, shape=[None, L, 1])
        self.prev_state = tf.placeholder(dtype=tf.float32, shape=[None, N])
        xv = xavier_initializer()
        ones = tf.initializers.ones()
        self.rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=N, activation=tf.nn.leaky_relu, name="" + str(
            random.random()))  # ,kernel_initializer=xv,bias_initializer=ones)
        self.wy = tf.Variable(xv(shape=[N, 1]), dtype=tf.float32)
        self.by = tf.Variable(ones(shape=[1, 1]), dtype=tf.float32)
        self.hidden_acts, self.states = tf.nn.dynamic_rnn(self.rnn_cell, inputs=self.input,
                                                          initial_state=self.prev_state)  # None x L x N
        self.out = tf.nn.leaky_relu(ndmatmul(self.hidden_acts, self.wy) + self.by)
        self.loss = tf.reduce_sum((self.out - self.desired) ** 2)
        opt = tf.train.AdamOptimizer()
        grads = opt.compute_gradients(self.loss)
        clipped = zip(tf.clip_by_global_norm([grad for grad, var in grads if grad != None], 10)[0],
                      [var for grad, var in grads if grad != None])
        self.train_ = opt.apply_gradients(clipped)
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def train(self):
        L = self.L
        sess = self.sess
        X = []
        Y = []
        for i in range(2 ** L):
            x = bin(i)[2:]
            x = "0" * (L - len(x)) + x
            x = list(map(int, x))
            y = [x[0]]
            for i in x[1:]:
                y.append(y[-1] * 2 + i)
            X.append(x)
            Y.append(y)
        indices = list(range(1, 2 ** L))
        random.shuffle(indices)
        X = [X[i] for i in indices]
        Y = [Y[i] for i in indices]
        Xs = partition(X, 50)
        Ys = partition(Y, 50)

        zero_state_0 = self.rnn_cell.zero_state(batch_size=len(Xs[0]), dtype=tf.float32).eval(session=sess)
        zero_state_1 = self.rnn_cell.zero_state(batch_size=len(Xs[-1]), dtype=tf.float32).eval(session=sess)
        for epoch in range(1000):
            total_loss = 0
            for X, Y in zip(Xs, Ys):
                zero_state = zero_state_0 if len(X) == len(Xs[0]) else zero_state_1
                _, loss, curr_states = sess.run([self.train_, self.loss, self.states],
                                                feed_dict={self.input: np.array(X).reshape([-1, L, 1]),
                                                           self.desired: np.array(Y).reshape([-1, L, 1]),
                                                           self.prev_state: zero_state})
                total_loss += loss
            if epoch % 100 == 0:
                mean_loss = total_loss / 2 ** L
                print(mean_loss)

    def run(self, seq):
        sess = self.sess
        L = self.L
        outputs = []
        prev_state = self.rnn_cell.zero_state(batch_size=1, dtype=tf.float32).eval(session=sess)
        while len(seq) >= L:
            s = seq[:L]
            inp = list(map(int, s))
            out, prev_state = self.sess.run([self.out, self.states],
                                            feed_dict={self.input: np.array(inp).reshape([1, -1, 1]),
                                                       self.prev_state: prev_state})
            outputs.extend(out.tolist()[0][:L])
            seq = seq[L:]
        if len(seq) > 0:
            s = seq + "0" * (L - len(seq))
            inp = list(map(int, s))
            out = self.sess.run([self.out],
                                feed_dict={self.input: np.array(inp).reshape([1, -1, 1]), self.prev_state: prev_state})[
                0]
            outputs.extend(out.tolist()[0][:len(seq)])
        return outputs


rnn = RNN()
rnn.train()
rnn.run("1000")

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


383.74114990234375
323.4237365722656
41.19600296020508
4.985841751098633
0.9714641571044922
0.12593123316764832
0.02036137878894806
0.0044591547921299934
0.0019453082932159305
0.0007638398092240095


[[0.9967873096466064],
 [1.9970052242279053],
 [4.0078349113464355],
 [7.992044925689697]]